In [4]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
##### finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/visp350scnn1/data_set.pkl')
data_set = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

results = {}
for key in data_set.keys():
    data, stim, run_speed, im_array = data_set[key]
    
    n_pcs = range(1, 100, 5)
    non_lin = [0]

    #empty dictionary of the best model we've fit for each cell
    best_model = {k: (-np.inf, None) for k in data.keys()}
    #make a dictionary of dictionaries to store the results
    scores_dict = {k: {} for k in data.keys()}
    bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

    for (l, n) in product(non_lin, n_pcs):
        bar.update()

        #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
        stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)

        for c in data.keys():
            X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
            weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]

            if l == 0:
                nonlin = tf.exp
            if l == 1:
                nonlin = tf.nn.sigmoid
            if l == 2:
                nonlin = tf.nn.relu


            model = gm.gaussian_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 1, reg = 'l2', 
                                    lr = 1e-5, train_params = False, scale_init = 1, bias_init = 0)

            loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                                 max_iters = 80) 

            
            _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
            _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

            scores_dict[c][(l, n)] = (r2_test, r2_train) 

            if r2_test > best_model[c][0]:
                best_model[c] = (r2_test, (l, n))
                print "new best model for cell ", c, ". r2 test: ", r2_test
                
        results[key] = best_model


0%                100%
[#                   ] | ETA: 00:00:01

new best model for cell  517398528 . r2 test:  -0.116098559626
new best model for cell  517397506 . r2 test:  0.03127161539
new best model for cell  517398020 . r2 test:  0.0347696378072
new best model for cell  517397398 . r2 test:  0.161674994398
new best model for cell  517398023 . r2 test:  0.0244427478601
new best model for cell  517398536 . r2 test:  0.0905584441836
new best model for cell  517398540 . r2 test:  0.055165325502
new best model for cell  517397517 . r2 test:  0.0497631379498
new best model for cell  517398030 . r2 test:  0.0414683063803
new best model for cell  517398544 . r2 test:  -0.0140621010119
new best model for cell  517398033 . r2 test:  0.18745679264
new best model for cell  517397522 . r2 test:  -0.00335664385202
new best model for cell  517398548 . r2 test:  0.0409354996812
new best model for cell  517397525 . r2 test:  0.0305801907425
new best model for cell  517398039 . r2 test:  0.00989418786787
new best model for cell  517397528 . r2 test:  0.06732605

In [5]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]
    

explained var:  0.201 non_lin: 0 n_components 1
(0.20059416142276593, (0, 1))
explained var:  0.062 non_lin: 0 n_components 106
(0.062190976895266627, (0, 106))
explained var:  0.045 non_lin: 0 n_components 96
(0.045422332587471401, (0, 96))
explained var:  0.347 non_lin: 0 n_components 6
(0.34680115764845965, (0, 6))
explained var:  0.342 non_lin: 0 n_components 1
(0.34246259761258385, (0, 1))
explained var:  0.215 non_lin: 0 n_components 1
(0.21488627103766758, (0, 1))
explained var:  0.063 non_lin: 0 n_components 1
(0.06317671337496894, (0, 1))
explained var:  0.023 non_lin: 0 n_components 81
(0.02266407806334449, (0, 81))
explained var:  0.199 non_lin: 0 n_components 6
(0.19880265232733121, (0, 6))
explained var:  0.3 non_lin: 0 n_components 36
(0.29987683960112749, (0, 36))
explained var:  0.149 non_lin: 0 n_components 1
(0.14938940225716857, (0, 1))
explained var:  0.254 non_lin: 0 n_components 1
(0.25399200489275703, (0, 1))
explained var:  0.088 non_lin: 0 n_components 1
(0.087

In [6]:
pkl_file = open('./boc/formatted/LN_RS_Exp', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()